In [1]:
import pandas as pd
import numpy as np

In [2]:
cons = pd.read_parquet("/uss/hdsi-prismdata/q2-ucsd-consDF.pqt")
cons = cons.drop(columns = ["credit_score"])
cons = cons.dropna()
acc = pd.read_parquet("/uss/hdsi-prismdata/q2-ucsd-acctDF.pqt")
txn = pd.read_parquet("/uss/hdsi-prismdata/q2-ucsd-trxnDF.pqt")
catmap = pd.read_csv("/uss/hdsi-prismdata/q2-ucsd-cat-map.csv")

### account type counts per consumer (ohe)

In [3]:
acc_counts = acc.groupby("prism_consumer_id")["account_type"].value_counts().unstack(fill_value=0)
acc_counts.columns.name = None
cons = cons.merge(acc_counts, left_on= 'prism_consumer_id', right_index=True)
cons

,prism_consumer_id,evaluation_date,DQ_TARGET,401K,AUTO,BROKERAGE,CASH MANAGEMENT,CD,CHECKING,CONSUMER,...,MONEYMARKET,MORTGAGE,OTHER,OVERDRAFT,PREPAID,RETIREMENT,ROTH,SAVINGS,STOCK PLAN,STUDENT
0,0,2021-09-01,0.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,1,2021-07-01,0.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,2,2021-05-01,0.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,3,2021-03-01,0.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,4,2021-10-01,0.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13995,13995,2022-01-22,0.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
13996,13996,2022-02-01,0.0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,3,0,0
13997,13997,2021-12-24,0.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
13998,13998,2022-01-30,0.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,2,0,0


In [13]:
sample_acc = acc[acc['prism_consumer_id'] == '100']
sample_acc

,prism_consumer_id,prism_account_id,account_type,balance_date,balance
8739,100,8739,SAVINGS,2022-12-31,1853.07
8740,100,8740,CHECKING,2021-04-30,802.40


In [14]:
sample_txn = txn[txn['prism_consumer_id'] == '100']
sample_txn

,prism_consumer_id,prism_transaction_id,category,amount,credit_or_debit,posted_date
1394258,100,1392246,3,1000.00,CREDIT,2022-08-04
1394259,100,1392247,3,1000.00,CREDIT,2022-08-18
1394260,100,1392248,6,2.24,CREDIT,2022-08-26
1394261,100,1392249,3,1000.00,CREDIT,2022-09-01
1394262,100,1392250,3,788.65,CREDIT,2022-09-15
...,...,...,...,...,...,...
1394375,100,1392363,0,416.16,DEBIT,2023-01-01
1394376,100,1392364,1,200.00,DEBIT,2023-01-03
1394377,100,1392365,4,15.25,DEBIT,2023-01-10
1394378,100,1392366,26,1000.00,DEBIT,2023-01-13


In [6]:
# # 1. store all balance dates
# sample_acc['balance_date'].tolist()

[datetime.date(2022, 12, 31), datetime.date(2021, 4, 30)]

In [23]:
import numpy as np
import pandas as pd

txn = sample_txn.copy()
acc = sample_acc.copy()

# -------------------- CHANGED: sort + datetimes --------------------
txn["posted_date"] = pd.to_datetime(txn["posted_date"])
acc["balance_date"] = pd.to_datetime(acc["balance_date"])

txn = txn.sort_values("posted_date").reset_index(drop=True)      # CHANGED
acc = acc.sort_values("balance_date").reset_index(drop=True)     # CHANGED
# ------------------------------------------------------------------

# -------------------- CHANGED: total (all-accounts) EOD anchor --------------------
# If acc has one row per account per date, this sums to total consumer balance per EOD date.
acc_total_eod = (
    acc[["balance_date", "balance"]]
      .dropna()
      .groupby("balance_date", as_index=False)["balance"]
      .sum()
      .rename(columns={"balance": "total_eod_balance"})
      .sort_values("balance_date")
)  # CHANGED
# -------------------------------------------------------------------------------

# signed amounts: CREDIT +, DEBIT -
txn["signed_amount"] = np.where(
    txn["credit_or_debit"].str.upper().eq("CREDIT"),
    txn["amount"],
    -txn["amount"]
)

# EOD anchor is end-of-day, so anchor must be strictly before txn time
txn["_asof_time"] = txn["posted_date"] - pd.Timedelta("1ns")

# Forward anchor: most recent total EOD balance strictly before txn
fwd = pd.merge_asof(
    txn,
    acc_total_eod.rename(columns={"balance_date": "anchor_date"}),
    left_on="_asof_time",
    right_on="anchor_date",
    direction="backward",
)

# Running consumer balance from the anchor + cumulative signed txns since anchor
fwd["consumer_balance"] = (
    fwd["total_eod_balance"]
    + fwd.groupby("anchor_date", dropna=False)["signed_amount"].cumsum()
)

# Back-solve for txns before first available EOD balance
need_bfill = fwd["total_eod_balance"].isna()
if need_bfill.any():
    bwd = pd.merge_asof(
        txn.loc[need_bfill].copy(),
        acc_total_eod.rename(columns={"balance_date": "next_date", "total_eod_balance": "next_total_eod"}),
        left_on="posted_date",
        right_on="next_date",
        direction="forward",
    )

    bwd = bwd.sort_values(["next_date", "posted_date"], ascending=[True, False])
    rev_cum = bwd.groupby("next_date")["signed_amount"].cumsum()
    bwd["consumer_balance"] = bwd["next_total_eod"] - (rev_cum - bwd["signed_amount"])

    fwd.loc[need_bfill, "consumer_balance"] = bwd.sort_index()["consumer_balance"].to_numpy()

out = fwd.drop(columns=["_asof_time"])


In [24]:
out

,prism_consumer_id,prism_transaction_id,category,amount,credit_or_debit,posted_date,signed_amount,anchor_date,total_eod_balance,consumer_balance
0,100,1392296,0,200.00,DEBIT,2022-08-01,-200.00,2021-04-30,802.40,602.40
1,100,1392266,0,200.00,CREDIT,2022-08-01,200.00,2021-04-30,802.40,802.40
2,100,1392297,26,901.70,DEBIT,2022-08-01,-901.70,2021-04-30,802.40,-99.30
3,100,1392324,0,485.72,DEBIT,2022-08-01,-485.72,2021-04-30,802.40,-585.02
4,100,1392325,0,416.16,DEBIT,2022-08-01,-416.16,2021-04-30,802.40,-1001.18
...,...,...,...,...,...,...,...,...,...,...
117,100,1392294,3,689.31,CREDIT,2023-01-20,689.31,2022-12-31,1853.07,2161.52
118,100,1392264,3,1535.29,CREDIT,2023-01-20,1535.29,2022-12-31,1853.07,3696.81
119,100,1392323,26,1714.25,DEBIT,2023-01-20,-1714.25,2022-12-31,1853.07,1982.56
120,100,1392295,6,1.36,CREDIT,2023-01-27,1.36,2022-12-31,1853.07,1983.92


In [7]:
# def balance by consumer

In [8]:
cons

,prism_consumer_id,evaluation_date,DQ_TARGET,401K,AUTO,BROKERAGE,CASH MANAGEMENT,CD,CHECKING,CONSUMER,...,MONEYMARKET,MORTGAGE,OTHER,OVERDRAFT,PREPAID,RETIREMENT,ROTH,SAVINGS,STOCK PLAN,STUDENT
0,0,2021-09-01,0.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,1,2021-07-01,0.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,2,2021-05-01,0.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,3,2021-03-01,0.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,4,2021-10-01,0.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13995,13995,2022-01-22,0.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
13996,13996,2022-02-01,0.0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,3,0,0
13997,13997,2021-12-24,0.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
13998,13998,2022-01-30,0.0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,2,0,0


In [9]:
acc

,prism_consumer_id,prism_account_id,account_type,balance_date,balance
0,3023,0,SAVINGS,2021-08-31,90.57
1,3023,1,CHECKING,2021-08-31,225.95
2,4416,2,SAVINGS,2022-03-31,15157.17
3,4416,3,CHECKING,2022-03-31,66.42
4,4227,4,CHECKING,2021-07-31,7042.90
...,...,...,...,...,...
24461,11500,24461,CHECKING,2022-03-27,732.75
24462,11615,24462,SAVINGS,2022-03-30,5.00
24463,11615,24463,CHECKING,2022-03-30,1956.46
24464,12210,24464,CHECKING,2022-03-28,2701.51


In [10]:
txn

,prism_consumer_id,prism_transaction_id,category,amount,credit_or_debit,posted_date
0,3023,0,4,0.05,CREDIT,2021-04-16
1,3023,1,12,481.56,CREDIT,2021-04-30
2,3023,2,4,0.05,CREDIT,2021-05-16
3,3023,3,4,0.07,CREDIT,2021-06-16
4,3023,4,4,0.06,CREDIT,2021-07-16
...,...,...,...,...,...,...
6407316,10533,6405304,31,4.96,DEBIT,2022-03-11
6407317,10533,6405305,12,63.48,DEBIT,2022-03-30
6407318,10533,6405306,12,53.99,DEBIT,2022-03-30
6407319,10533,6405307,12,175.98,DEBIT,2022-03-31


if savings anchor is 12-31-2022 2000, should the savings balance always be 2000 before the anchor date or add 2000 to total balance on anchor date